# Twitter-Sentiment-Analysis-ML_Model (Soumyadip Majumder)

In [3]:
# installing kaggle library
!pip install kaggle

In [4]:
# configuring the path of path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## Importing Twitter Sentiment dataset

In [5]:
# API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 98%|█████████████████████████████████████ | 79.0M/80.9M [00:04<00:00, 26.8MB/s]
100%|██████████████████████████████████████| 80.9M/80.9M [00:04<00:00, 20.3MB/s]


## Importing the Dependencies

In [6]:
!pip install scikit-learn
!pip install pandas
!pip install nltk

## Importing Libraries

In [7]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:
# checking the number of rows and columns
row,col = twitter_data.shape
print("Rows: ", row)
print("Columns: ", col)

Rows:  1599999
Columns:  6


In [11]:
# naming the columns and reading the dataset again

column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('training.1600000.processed.noemoticon.csv', names=column_names, encoding='ISO-8859-1')
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
twitter_data.shape

(1600000, 6)

In [12]:
# counting the number of missing values in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

###### From this analysis
There is no null value in this dataset.

In [14]:
# checking the distribution of target column
twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

From this output, we can say that this dataset contains **16M** tweets extracted using _twitter API_. The tweets have been annotated (0 means negative value and 4 means positive value)

##### From the analysis process ('target' column)
_total 8M is negative_

_total 8M is positive_

#### Convert the target "4" to "1" for good analysis

In [15]:
twitter_data.replace({'target':{4:1}}, inplace=True)
print("Replaced the all values...")

Replaced the all values...


#### After replacing the the all values
##### 0 -> Negative Tweet
##### 1 -> Positive Tweet

##### Stemming is the process to reduce a word to its Root word. It makes for visible and more clear for updated dataset.

#### port_stem = PorterStemmer()
###### This line initializes an instance of the PorterStemmer class from the nltk library. The Porter Stemmer is a popular algorithm for stemming words, which means reducing words to their root form. For example, “running” becomes “run”.

In [16]:
port_stem = PorterStemmer()

##### <mark>def stemming(content):</mark>
This line defines a function named `stemming` that takes a single argument `content`.


`stemmed_content = re.sub('[^a-zA-Z]',' ', content)` --> This line uses the `re` _(regular expression)_ module to substitute any character in `content` that is not a letter (a-z or A-Z) with a space. This helps in removing punctuation, number, and special characters from the text.

`streamed content = stemmed_content.split()` --> This line splits the `stemmed_content` string into a list of words. The default delimiter is any white-space.

<br><br>
`stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]` --> This line performs several actions:
###### 1. List Comprehension:
It iterates over each `word` in the `stemmed_content` list.
<br>
###### 2. Stemming:
For each, `word`, it applies the `stem` method of the `PorterStemmer` instance _(port_stem)_ to reduce the word to its root form.
<br>
###### 3. Stop-words Removal:
It includes the word in the final list only if it is not in the list of English stopwords. Stopwords are common words like "and", "the", "is", etc., which are often removed in text processing because they don't carry significant meaning.


<br><br>

`stemmed_content = ' '.join(stemmed_content)` -->
This line joins the list of stemmed words back into a single string, with each word separated by a space.

<br>

`return stemmed_content` --> simply return the stemmed value.



##### Full Function Explanation
The `stemming` function takes a text input, removes non-alphabetic characters, converts the text to lowercase, splits it into words, stems each word (excluding stopwords), and then joins the words back into a single string. This processed string is then returned.

In [17]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

##### Feature Engineering
###### In this dataset there are total of 16 million rows, so after running the `feature engineering` cell block it will take a lot time to execute.
###### _Soumyadip Majumder_
###### _Kernel: AzureML <mark>(Python 3.8)</mark>_

In [18]:
# feature engineering
# adding new column
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [19]:
# checking for updated dataset
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [20]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [21]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [22]:
# separating the data and label
X = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [23]:
print("X: ", X)
print("y: ", y)

X:  ['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']
y:  [0 0 0 ... 1 1 1]


In [24]:
# split the data for training and test the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

In [25]:
print("X.shape: ", X.shape)
print("X_train.shape: ", X_train.shape)
print("X_test.shape: ", X_test.shape)

X.shape:  (1600000,)
X_train.shape:  (1280000,)
X_test.shape:  (320000,)


In [26]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [27]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


###### Converting the textual data to numerical data

In [28]:
# converting text to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [29]:
print(X_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.2725987626483838
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.20289715703997938
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.1870933868497303
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.2206474219107611
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.22365675600992338
  (1279996, 29107

In [30]:
print(X_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264266
  (0, 388348)	0.21985076072061732
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.2680037527082731
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498775
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.45303413825598426
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

#### Training the Machine Learning Model
##### Logistic Regression

In [32]:
model = LogisticRegression(max_iter=1000)

In [33]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

#### Model Evaluation
##### Accuracy score

In [35]:
# accuracy score on the trainning data

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(y_train, X_train_prediction)

In [36]:
print('Accuracy score on the training data: ', training_data_accuracy)

Accuracy score on the training data:  0.81023671875


In [39]:
print('Accuracy in percentage: {:.2f}%'.format(training_data_accuracy*100))

Accuracy in percentage: 81.02%


In [40]:
# accuracy score on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(y_test, X_test_prediction)

In [51]:
# For better visualisation I've created a custom formatted display

from IPython.display import display, Markdown

def display_markdown_test_dataOutput(output):
    display(Markdown(output))

formatted_test_data_accuracy = f"**Test data accuracy: {test_data_accuracy*100:.2f}%**"
display_markdown_test_dataOutput(formatted_test_data_accuracy)

**Test data accuracy: 77.80%**

#### Saving the trained model

In [52]:
import pickle

In [53]:
filename = 'trained_model.pkl'
pickle.dump(model, open(filename, 'wb'))

##### Using the saved model for future predictions

In [54]:
# loading the saved model
loaded_model = pickle.load(open('trained_model.pkl', 'rb'))   # 'rb': Read in Binary format

In [57]:
X_new = X_test[200]
print(y_test[200])

new_prediction = model.predict(X_new)
print(new_prediction)

if (new_prediction[0] == 0):
    print('Negative Tweet')
else:
    print('Positive Tweet')

1
[1]
Positive Tweet


##### Complete AzureML-Tweet-Sentiment-Analysis

**Soumyadip Majumder**

**Kernel: AzureML (Python3.8)**